<a href="https://colab.research.google.com/github/yuji-sgs/m1-reserch/blob/main/MOPSO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# はじめに

## 本ノートブックの目的
[前回作成したノートブック](https://github.com/yuji-sgs/m1-reserch/blob/main/Inverse_Analysis.ipynb)では深層学習モデルの1出力に対する逆解析にしか適応できなかったが、複数出力に対する逆解析を実現するために、KerasのFunctional APIモデルを用いて複数出力の深層学習モデルを作成し、その学習済みモデルを適用して多目的粒子群最適化（MOPSO）アルゴリズムを構築していく。

## 処理フロー
1. 使用するライブラリをインポート
2. データの読み込み・確認
3. データ分割
4. KerasのFunctional APIモデルで学習
5. 学習後の精度評価
6. MOPSO実装

## 1. 使用するライブラリをインポート

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from scipy.interpolate import griddata

import io
from google.colab import files

import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Activation
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, LearningRateScheduler
from keras.models import Model
from keras.layers import Input, Dense

from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt
from sklearn.metrics import r2_score

## 2. データの読み込み・確認
ローカルにあるファイルからGoogle Colaboratoryにアップロード

In [2]:
# Google Colaboratoryでファイルをアップロード
uploaded = files.upload()

Saving data_Multiple.csv to data_Multiple.csv


In [3]:
# データフレームとして、変数dfに4列×10000行のデータを格納（C2：材料パラメータ, s：構造パラメータ, w11_max：遮音パラメータ1,　BandGap1：遮音パラメータ2）
df = pd.read_csv(io.BytesIO(uploaded['data_Multiple.csv']))
df

,C2,s,w11_max,BandGap1
0,0.15,0.01,250.00,6.689500e+00
1,0.30,0.01,353.55,5.649800e+00
2,0.45,0.01,433.01,4.264400e+00
3,0.60,0.01,500.00,2.945800e+00
4,0.75,0.01,559.02,1.748300e+00
...,...,...,...,...
9995,14.40,1.00,645.50,4.090000e-12
9996,14.55,1.00,645.50,7.840000e-12
9997,14.70,1.00,645.50,2.270000e-13
9998,14.85,1.00,645.50,4.550000e-12


## 3. データ分割
学習用データとテスト用データに分割

In [4]:
# データの分割
(train, test) = train_test_split(df, test_size=0.2, shuffle=True)

In [5]:
# x_train, x_testではcsvファイルの1, 2列目を， y_train, y_testでは3, 4列目のデータを活用
x_train = train.iloc[:, [0, 1]]
y_train = train.iloc[:, [2, 3]]

x_test = test.iloc[:, [0, 1]]
y_test = test.iloc[:, [2, 3]]